<a href="https://colab.research.google.com/github/Eslam-Mohamed10/Code_Alpha_Machine_learning_Diesese/blob/main/Code_Alpha_Machine_learning_Diesese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

In [3]:
dataset = pd.read_csv('diabetes.csv')

In [4]:
dataset.shape

(768, 9)

In [5]:
dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [7]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [8]:
dataset.isna().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [9]:
 ## Specifying categorical and numerical data seperately.
categorical_features = [column for column in dataset.columns if dataset[column].dtype=='O']
numeric_features = [column for column in dataset.columns if dataset[column].dtype!='O']

In [10]:
print('This are the columns (features) with Object data type: ', categorical_features)
print('This are the columns (features) with Non-Object data type: ', numeric_features)

This are the columns (features) with Object data type:  []
This are the columns (features) with Non-Object data type:  ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']


In [11]:
def find_cat(data):
    for col in data.columns:
        print(col,':',data[col].nunique())

In [12]:
find_cat(dataset)

Pregnancies : 17
Glucose : 136
BloodPressure : 47
SkinThickness : 51
Insulin : 186
BMI : 248
DiabetesPedigreeFunction : 517
Age : 52
Outcome : 2


In [13]:
## Now we will select features for X and Y.
X = dataset.drop(columns =['Outcome'],axis=1)
Y = dataset['Outcome']

In [14]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [15]:
Y

,Outcome
0,1
1,0
2,1
3,0
4,1
...,...
763,0
764,0
765,0
766,1


In [16]:
## Now we will  split X,Y to training set and testing set.
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [17]:
print(dataset.std())

Pregnancies                   3.369578
Glucose                      31.972618
BloodPressure                19.355807
SkinThickness                15.952218
Insulin                     115.244002
BMI                           7.884160
DiabetesPedigreeFunction      0.331329
Age                          11.760232
Outcome                       0.476951
dtype: float64


In [18]:
scaler = StandardScaler()
# X = scaler.fit_transform(X)

scaler.fit(X_train)
# X_train_standardizer = scaler.transform(X_train)
X_train_standardizer = scaler.fit_transform(X_train)
X_test_standardizer = scaler.transform(X_test)

In [19]:
print(X_train_standardizer.std())

1.0


In [20]:
print(X_test_standardizer.std())

1.06887872449587


In [21]:
from sklearn.linear_model import LogisticRegression
regressor = LogisticRegression(max_iter=1000)

In [22]:
regressor.fit(X_train_standardizer,Y_train)

LogisticRegression(max_iter=1000)

In [23]:
X_train_prediction = regressor.predict(X_train_standardizer)
X_train_accuracy = accuracy_score(X_train_prediction,Y_train)

In [24]:
print("Accuracy score of model is:",X_train_accuracy)

Accuracy score of model is: 0.7703583061889251


In [25]:
X_test_prediction = regressor.predict(X_test_standardizer)
X_test_accuracy = accuracy_score(X_test_prediction,Y_test)

In [26]:
print("Accuracy score of model is:",X_test_accuracy)

Accuracy score of model is: 0.7532467532467533


In [27]:
cv_score_lr = cross_val_score(LogisticRegression(max_iter=1000), X, Y, cv=5)

print(cv_score_lr)

mean_accuracy_lr = sum(cv_score_lr)/len(cv_score_lr)

mean_accuracy_lr = mean_accuracy_lr*100

mean_accuracy_lr = round(mean_accuracy_lr, 2)

print(mean_accuracy_lr)

[0.77272727 0.74675325 0.75324675 0.81045752 0.77777778]
77.22


In [28]:
models = {
    'LogisticRegression':LogisticRegression(
        max_iter=1000
    ),
    'SVC':SVC(
        kernel='rbf',  # Radial Basis Function kernel
        C=1.0,         # Regularization parameter
        gamma='scale'  # Kernel coefficient ('auto' for older versions)
    ),
    'DecisionTree':DecisionTreeClassifier(
        criterion="entropy",  # or "gini" for impurity measure
        max_depth=3,          # control tree depth to prevent overfitting
        random_state=42
    ),
    'XGBoost':XGBClassifier(
        n_estimators=100,
        max_depth=3,
        learning_rate=0.1,
        objective='binary:logistic')
}

In [29]:
def compare_models_train_test():

  for model in models:

    # training the model
    model.fit(X_train, Y_train)

    # evaluating the model
    test_data_prediction = model.predict(X_test)

    accuracy = accuracy_score(Y_test, test_data_prediction)

    print('Accuracy score of the ', model, ' = ', accuracy)

In [30]:
# Train and evaluate each model
for model_name, model in models.items():
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(Y_test, y_pred)

    print(f"Model: {model_name}\n, Accuracy_Score: {accuracy}")
    print(classification_report(Y_test, y_pred))

Model: LogisticRegression
, Accuracy_Score: 0.7467532467532467
              precision    recall  f1-score   support

           0       0.81      0.79      0.80        99
           1       0.64      0.67      0.65        55

    accuracy                           0.75       154
   macro avg       0.73      0.73      0.73       154
weighted avg       0.75      0.75      0.75       154

Model: SVC
, Accuracy_Score: 0.7662337662337663
              precision    recall  f1-score   support

           0       0.78      0.88      0.83        99
           1       0.72      0.56      0.63        55

    accuracy                           0.77       154
   macro avg       0.75      0.72      0.73       154
weighted avg       0.76      0.77      0.76       154

Model: DecisionTree
, Accuracy_Score: 0.7662337662337663
              precision    recall  f1-score   support

           0       0.81      0.84      0.82        99
           1       0.69      0.64      0.66        55

    accuracy  

In [31]:
def compare_models_cross_validation():
    for model_name, model in models.items():
        cv_score = cross_val_score(model, X,Y, cv=5)

        mean_accuracy = sum(cv_score)/len(cv_score)

        mean_accuracy = mean_accuracy*100

        mean_accuracy = round(mean_accuracy, 2)

        print('Cross Validation accuracies for ', model, '=  ', cv_score)
        print('Accuracy % of the ', model, mean_accuracy)
        print('----------------------------------------------')

In [32]:
xgbc = XGBClassifier(n_estimators=100,max_depth=3,learning_rate=0.1,objective='binary:logistic')
xgbc.fit(X_train_standardizer,Y_train)

X_test_prediction = xgbc.predict(X_test_standardizer)
X_test_accuracy = accuracy_score(X_test_prediction,Y_test)

In [33]:
X_test_accuracy

0.7337662337662337

In [34]:
cv_score_lr = cross_val_score(XGBClassifier(n_estimators=100,max_depth=3,learning_rate=0.1,objective='binary:logistic'), X, Y, cv=5)

print(cv_score_lr)

mean_accuracy_lr = sum(cv_score_lr)/len(cv_score_lr)

mean_accuracy_lr = mean_accuracy_lr*100

mean_accuracy_lr = round(mean_accuracy_lr, 2)

print(mean_accuracy_lr)

[0.77922078 0.75324675 0.74675325 0.84313725 0.74509804]
77.35


In [35]:
input_data = (5,166,72,19,175,25.8,0.587,51)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)

prediction = xgbc.predict(std_data)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[[ 0.37979288  1.41038486  0.13973176 -0.09071957  0.80559475 -0.79947408
   0.35009487  1.57410227]]
[1]
The person is diabetic
